# Search engine title completion with GPT-2

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b621b819-3704-2881-0e9e-ecd8a8f938a2)


In [2]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 17.7 MB/s 
     |████████████████████████████████| 6.6 MB 50.2 MB/s 
     |████████████████████████████████| 101 kB 6.8 MB/s 


In [3]:
import pandas as pd
import regex as re

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
posts = pd.read_csv('drive/MyDrive/redditnlp_0na.csv',index_col=0)

In [43]:
posts.head()

,id,title,subreddit,body,num_comments,all_text
0,wopny7,Weekly Entering & Transitioning - Thread 15 Au...,datascience,\n\nWelcome to this week's entering & transit...,50,Weekly Entering & Transitioning - Thread 15 Au...
1,wqp4qm,When you are invited to a ‘town hall’ amidst a...,datascience,NaN,85,When you are invited to a ‘town hall’ amidst a...
2,wrd1s7,Resume critique (trying to get an internship o...,datascience,NaN,13,Resume critique (trying to get an internship o...
3,wrc8ad,Advice - Looking for a Data Scientist,datascience,Been trying to find a solid Data Scientist for...,51,Advice - Looking for a Data Scientist Been try...
4,wriszy,Recommendations for Udemy class on deploying a...,datascience,I use Python mostly. Trying to move from analy...,0,Recommendations for Udemy class on deploying a...


### Finding if any missing values

In [44]:
posts.isna().any()

id              False
title           False
subreddit       False
body             True
num_comments    False
all_text        False
dtype: bool

In [45]:
posts.fillna(' ',inplace=True)

In [46]:
posts.head()

,id,title,subreddit,body,num_comments,all_text
0,wopny7,Weekly Entering & Transitioning - Thread 15 Au...,datascience,\n\nWelcome to this week's entering & transit...,50,Weekly Entering & Transitioning - Thread 15 Au...
1,wqp4qm,When you are invited to a ‘town hall’ amidst a...,datascience,,85,When you are invited to a ‘town hall’ amidst a...
2,wrd1s7,Resume critique (trying to get an internship o...,datascience,,13,Resume critique (trying to get an internship o...
3,wrc8ad,Advice - Looking for a Data Scientist,datascience,Been trying to find a solid Data Scientist for...,51,Advice - Looking for a Data Scientist Been try...
4,wriszy,Recommendations for Udemy class on deploying a...,datascience,I use Python mostly. Trying to move from analy...,0,Recommendations for Udemy class on deploying a...


## Our goal is to complete text while querying a search engine

In [47]:
posts['content'] = posts['title'] +' '+'<|endoftitle|>'+'\n'+ posts['body']+' '+'<|endoftext|>'

In [48]:
posts.content.head().values

array(["Weekly Entering & Transitioning - Thread 15 Aug, 2022 - 22 Aug, 2022 <|endoftitle|>\n \n\nWelcome to this week's entering & transitioning thread! This thread is for any questions about getting started, studying, or transitioning into the data science field. Topics include:\n\n* Learning resources (e.g. books, tutorials, videos)\n* Traditional education (e.g. schools, degrees, electives)\n* Alternative education (e.g. online courses, bootcamps)\n* Job search questions (e.g. resumes, applying, career prospects)\n* Elementary questions (e.g. where to start, what next)\n\nWhile you wait for answers from the community, check out the [FAQ](https://www.reddit.com/r/datascience/wiki/frequently-asked-questions) and Resources pages on our wiki. You can also search for answers in [past weekly threads](https://www.reddit.com/r/datascience/search?q=weekly%20thread&restrict_sr=1&sort=new). <|endoftext|>",
       'When you are invited to a ‘town hall’ amidst a recession <|endoftitle|>\n  <|en

In [49]:
posts.head()

,id,title,subreddit,body,num_comments,all_text,content
0,wopny7,Weekly Entering & Transitioning - Thread 15 Au...,datascience,\n\nWelcome to this week's entering & transit...,50,Weekly Entering & Transitioning - Thread 15 Au...,Weekly Entering & Transitioning - Thread 15 Au...
1,wqp4qm,When you are invited to a ‘town hall’ amidst a...,datascience,,85,When you are invited to a ‘town hall’ amidst a...,When you are invited to a ‘town hall’ amidst a...
2,wrd1s7,Resume critique (trying to get an internship o...,datascience,,13,Resume critique (trying to get an internship o...,Resume critique (trying to get an internship o...
3,wrc8ad,Advice - Looking for a Data Scientist,datascience,Been trying to find a solid Data Scientist for...,51,Advice - Looking for a Data Scientist Been try...,Advice - Looking for a Data Scientist <|endoft...
4,wriszy,Recommendations for Udemy class on deploying a...,datascience,I use Python mostly. Trying to move from analy...,0,Recommendations for Udemy class on deploying a...,Recommendations for Udemy class on deploying a...


## Function to remove Urls:

In [50]:
def remove_links(corpus):
    new_body = []
    for i in corpus:
        new_body.append(re.sub(r'http\S+', '', i))
    return new_body

## Text completion on corpus

In [51]:
corpus = posts.content.values

In [52]:
corpus = remove_links(corpus)

In [53]:
len(corpus)

5729

## Make train and test set for ds_content
#### 5500 train and rest test

In [54]:
dataset_train = corpus[:5500]
dataset_test = corpus[5500:]


In [55]:
len(dataset_train)

5500

In [56]:
len(dataset_test)

229

## Load the basic pretrained GPT-2

In [57]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

## Load a finetuned model we previously saved

In [ ]:
import gdown
gdrivelink='https://drive.google.com/drive/folders/10iTwjGWN_zA_Zgbj8aSKIt8vWtn_2V95?usp=sharing' # this directory contains the fine-trained model
gdown.download_folder(gdrivelink, quiet=True) 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('')#model path
model = AutoModelForCausalLM.from_pretrained('')#model path

## Test model on one sentence

In [58]:
input_text = "Completely and totally 'burnt out' at work, unsure if I should quit or just go rouge and do my own core data science"
enc_input = tokenizer.encode(input_text,return_tensors='pt',add_special_tokens=False)
output_sequences = model.generate(
    input_ids = enc_input,
    max_length= 70,  # the length of the final sentence
    temperature = 0.9, # the closer to one, the less deterministic. The closer to zero, the more deterministic
    top_k = 20, # how many next words to consider when doing a tree-like structure
    top_p = 0.9,
    repetition_penalty = 1, # penalty for repeating a word in the input (min 1)
    do_sample = True, # True -> probabilistic model (output varies)
    num_return_sequences = 5 # number of output sentences
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [59]:
for i in range(len(output_sequences)):
  print(f'{i}: {tokenizer.decode(output_sequences[i])}\n')

0: Completely and totally 'burnt out' at work, unsure if I should quit or just go rouge and do my own core data science, and then go home and just enjoy my life, and then get ready for another life...























1: Completely and totally 'burnt out' at work, unsure if I should quit or just go rouge and do my own core data science, then I should get my job done! I'd love to see you all in the comments!<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>

2: Completely and totally 'burnt out' at work, unsure if I should quit or just go rouge and do my own core data science, I don't think I can do that.




But I'm a little more interested in the data, and I'd like to give you a little bit of perspective on it.

3: Completely and totally 'burnt out' at work, u

### The Pretrained model, although did generate some meaningful words, finetunig the model might perform better.

## Convert train and test data to txt

In [60]:
dataset_train[1]

'When you are invited to a ‘town hall’ amidst a recession <|endoftitle|>\n  <|endoftext|>'

In [61]:
with open('dataset_train.txt','w') as f:
  f.write('\n'.join(dataset_train))
with open('dataset_test.txt','w') as f:
  f.write('\n'.join(dataset_test))

In [62]:
!head -20 dataset_train.txt

Weekly Entering & Transitioning - Thread 15 Aug, 2022 - 22 Aug, 2022 <|endoftitle|>
 

Welcome to this week's entering & transitioning thread! This thread is for any questions about getting started, studying, or transitioning into the data science field. Topics include:

* Learning resources (e.g. books, tutorials, videos)
* Traditional education (e.g. schools, degrees, electives)
* Alternative education (e.g. online courses, bootcamps)
* Job search questions (e.g. resumes, applying, career prospects)
* Elementary questions (e.g. where to start, what next)

While you wait for answers from the community, check out the [FAQ]( and Resources pages on our wiki. You can also search for answers in [past weekly threads]( <|endoftext|>
When you are invited to a ‘town hall’ amidst a recession <|endoftitle|>
  <|endoftext|>
Resume critique (trying to get an internship or an entry level job) <|endoftitle|>
  <|endoftext|>
Advice - Looking for a Data Scientist <|endoftitle|>
Been trying to find a s

## Fine tune the model

In [63]:
!curl https://raw.githubusercontent.com/huggingface/transformers/27c1b656cca75efa0cc414d3bf4e6aacf24829de/examples/run_lm_finetuning.py > run_lm_finetuning.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31078  100 31078    0     0   333k      0 --:--:-- --:--:-- --:--:--  337k


In [65]:
!rmdir experiments
!mkdir experiments

rmdir: failed to remove 'experiments': No such file or directory


In [66]:
epochs=3
file_with_training_set = 'dataset_train.txt'

In [67]:
# this is bash code to fine-tune a language model. File from here: https://github.com/alontalmor/pytorch-transformers/blob/master/examples/run_lm_finetuning.py
text = f"for epoch in {epochs} \n"+\
"do \n"+\
"python run_lm_finetuning.py "+\
f"--output_dir=experiments/epoch_{epochs} "+\
"--model_type=gpt2 "+\
"--model_name_or_path=distilgpt2 "+\
f"--train_data_file={file_with_training_set} "+\
"--do_train "+\
"--overwrite_output_dir "+\
"--save_steps=200 " +\
f"--num_train_epochs={epochs} \n" +\
"done"

In [68]:
f = open('run_experiments.sh',mode='w')
f.write(text)
f.close()

In [69]:
!bash run_experiments.sh

08/21/2022 20:36:51 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/21/2022 20:36:57 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-08, block_size=1024, cache_dir='', config_name='', device=device(type='cuda'), do_eval=False, do_lower_case=False, do_train=True, eval_all_checkpoints=False, eval_data_file=None, evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=1, learning_rate=5e-05, local_rank=-1, logging_steps=50, max_grad_norm=1.0, max_steps=-1, mlm=False, mlm_probability=0.15, model_name_or_path='distilgpt2', model_type='gpt2', n_gpu=1, no_cuda=False, num_train_epochs=3.0, output_dir='experiments/epoch_3', overwrite_cache=False, overwrite_output_dir=True, per_gpu_eval_batch_size=4, per_gpu_train_batch_size=4, save_steps=200, save_total_limit=None, seed=42, server_ip='', server_port='', tokenizer_name='', train_data_file='dataset_train.t

## Save the Model

In [71]:
tokenizer = AutoTokenizer.from_pretrained('experiments/epoch_3')
model = AutoModelForCausalLM.from_pretrained('experiments/epoch_3')

model_path = "Text_completion_on_reddit_submissions"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
# mount it
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
# copy it as a new directory in the root of your google drive
import shutil
shutil.copytree(model_path,'/content/drive/MyDrive/NLP'+ model_path)

Mounted at /content/drive


'/content/drive/MyDrive/NLPText_completion_on_reddit_submissions'

## Testing

In [75]:
dataset_test[189]

"[P] Real-time AR for jewelry virtual try on that looks real, done with joliGAN, based on a few 2D videos and no 3D model <|endoftitle|>\nA work from us with GANs recently emerged from stealth [\n\nThe hands are real, but the rings are rendered with a GAN in real-time. A first network detects where to render the ring, a second network does the rendering. There's no 3D model, it's purely 2D to 2D.\n\n&#x200B;\n\n\n\nWe thought we'd share some technical details since the underlying code, JoliGAN is Open Source, [\n\n\\- The GAN uses a combination of mobile ResNets with attention as a Generator, along with a projected Discriminator \\[1\\]. Depending on the stone, we sometimes use transformers as well (customized Segformers and ViT mostly). A series of additional neural networks act as semantic constraints to the space of GAN transforms.\n\n\\- Real-time is achieved through our full C++ Open Source backend DeepDetect, [ We use CUDA along with OpenCV and TensorRT to chain multiple models (

In [76]:
input_text = dataset_test[189].split('<|endoftitle|>')[0]+'\nGPT-2 generated text:\n'

In [77]:
enc_input = tokenizer.encode(input_text,return_tensors='pt',add_special_tokens=False)
output_sequences = model.generate(
    input_ids = enc_input,
    max_length= 300,  # the length of the final sentence
    temperature = 0.9, # the closer to one, the less deterministic. The closer to zero, the more deterministic
    top_k = 20, # how many next words to consider when doing a tree-like structure
    top_p = 0.9,
    repetition_penalty = 1, # penalty for repeating a word in the input (min 1)
    do_sample = True, # True -> probabilistic model (output varies)
    num_return_sequences = 3 # number of output sentences
)
for i in range(len(output_sequences)):
  print(f'{i}: {tokenizer.decode(output_sequences[i])}\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: [P] Real-time AR for jewelry virtual try on that looks real, done with joliGAN, based on a few 2D videos and no 3D model 
GPT-2 generated text:

[https://www.youtube.com/watch?v=8jYt6vVf7m](https://www.youtube.com/watch?v=8jYt6vVf7m)

The images are of the original model, and are also from the GPT-2 version of the original model. 

[https://arxiv.org/abs/2206.09092](https://arxiv.org/abs/2206.092)

This is a very powerful and useful tool to use in real-time AR. 

[https://www.youtube.com/watch?v=8jYt6vvf7m](https://www.youtube.com/watch?v=8jYt6vf7m)

I'm curious if you can use it to generate a real-time image of your house, or if you would like to use it to make it real-time with the built-in joliGAN model. <|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endof

#### The model may not be doing a great job completing the body when title is given but there is a clear improvement in performance since the pretrained model as it is generating related links to the content in title. Probably more data could help improve the performance significantly.
